In [1]:
import cobra
import libsbml
from cobra.core import Metabolite, Reaction
import pandas as pd
from cobra import flux_analysis
ModelF = cobra.io.read_sbml_model("CAM_12P_ME_Model.xml")
ModelF.solver="glpk"
cobra.flux_analysis.pfba(ModelF)

4.450782702947559
-------
0.42999264936692144
-------
Phloem_output_tx1	-0.3097224281318271
0.0022970297 4_AMINO_BUTYRATE_c1 + 0.0004186704 ARG_c1 + 0.0015049505 ASN_c1 + 0.0004186704 CYS_c1 + 0.0792079208 FRU_c1 + 0.0693069307 GLC_c1 + 0.0240792079 GLN_c1 + 0.0124356436 GLT_c1 + 0.0007128713 GLY_c1 + 0.0004186704 HIS_c1 + 0.0017425743 ILE_c1 + 0.0020594059 LEU_c1 + 0.0022178218 LYS_c1 + 0.0038811881 L_ALPHA_ALANINE_c1 + 0.006019802 L_ASPARTATE_c1 + 0.0004186704 MET_c1 + 0.0057029703 PHE_c1 + 0.9603960396 PROTON_e1 + 0.0004186704 PRO_c1 + 0.003960396 SER_c1 + 0.0068910891 THR_c1 + 0.0004186704 TRP_c1 + 0.0004186704 TYR_c1 + 0.0027722772 VAL_c1 + 0.7326732673 sSUCROSE_b1 --> 0.9603960396 PROTON_c1 + Phloem_e1
-------
PROTON_ATPase_c1	0.3105325415071109
0.65 ATP_c1 + 0.45 PROTON_c1 + WATER_c1 + 0.35 aATP_c1 --> 0.5 ADP_c1 + PROTON_e1 + 0.7 Pi_c1 + 0.5 aADP_c1 + 0.3 aPi_c1
-------
5.8315578352997575
-------
0.35750555178863164


In [2]:
ModelF3 = ModelF.copy()

In [3]:
#Sensitivity analysis - Rubisco carboxylase/oxygenase ratio

#Constrain different Rubisco carboxylase and oxygenase ratios (Vc/Vo) (examined ratios: 2, 3, 5, 10, 1000)
# Vc/Vo = 2 (change 2 to the ratio that needs to be checked)
for i in range(1,7):
    new_constraint3 = ModelF3.problem.Constraint(
        2*ModelF3.reactions.get_by_id("RXN_961_p"+str(i)).flux_expression - ModelF3.reactions.get_by_id("RIBULOSE_BISPHOSPHATE_CARBOXYLASE_RXN_p"+str(i)).flux_expression,
        lb=0,
        ub=0)
    ModelF3.add_cons_vars(new_constraint3)

    sol3 = flux_analysis.parsimonious.pfba(ModelF3)
    
#@Sanu: Add function to derive top 5 day/night proton consuming/producing?

In [1]:
#Sensitivity analysis - Maintencance cost

#adding maintenance cost --> @Sanu: needed here? As maintenance cost was added in a previous model already
PPFD = 100                            #light intensity of the model
ATPase = (0.0049*PPFD) + 2.7851      #non-growth assocaited maintenance (NGAM) cost based on light - see Topfer et al 2020 Supplemental information section 1.2.3

#Uncomment to check flux distribution
#ATPase = ((0.0049*PPFD) + 2.7851)*0.5 #Decrease maintenance cost by 50%
#ATPase = ((0.0049*PPFD) + 2.7851)*1.5 #Increase maintenance cost by 50%

for i in range(1,13):
    ModelF.reactions.get_by_id("ATPase_tx"+str(i)).lower_bound = ATPase
    ModelF.reactions.get_by_id("ATPase_tx"+str(i)).upper_bound = ATPase
    
    #Setting NADPH demand to 1/3 of ATP demand and distributing this demand to cytosol, plastid and mitochondria based on Cheung et al 2013 (doi: 10.1111/tpj.12252)
    ModelF.reactions.get_by_id("NADPHoxc_tx"+str(i)).lower_bound = ATPase/9
    ModelF.reactions.get_by_id("NADPHoxc_tx"+str(i)).upper_bound = ATPase/9
    
    ModelF.reactions.get_by_id("NADPHoxp_tx"+str(i)).lower_bound = ATPase/9
    ModelF.reactions.get_by_id("NADPHoxp_tx"+str(i)).upper_bound = ATPase/9
    
    ModelF.reactions.get_by_id("NADPHoxm_tx"+str(i)).lower_bound = ATPase/9
    ModelF.reactions.get_by_id("NADPHoxm_tx"+str(i)).upper_bound = ATPase/9
        
cobra.flux_analysis.pfba(ModelF)

sol3 = flux_analysis.parsimonious.pfba(ModelF3)

#@Sanu: Add function to derive top 5 day/night proton consuming/producing?

NameError: name 'ModelF' is not defined

In [ ]:
#Sensitivity analysis - Decarboxylating enzyme
#@Sanu: Remove constraint PEPCK=0 and ME carries 0 flux automatically? (the flux ratio constraints on NAD-ME/NADP-ME and cPPDK and pPPDK also need to be removed (see notebook ME-refinement))   
  
#@Sanu: how to re-activate PEPCK in the model and get rid of flux ratio constrains on ME and PPDK? 
# for i in range(1,13):    
#     ModelF3.reactions.get_by_id("PEPCARBOXYKIN_RXN_c"+str(i)).lower_bound = 0
#     ModelF3.reactions.get_by_id("PEPCARBOXYKIN_RXN_c"+str(i)).upper_bound = 0
    
sol3 = flux_analysis.parsimonious.pfba(ModelF3)

#@Sanu: Add function to derive top 5 day/night proton consuming/producing?

In [23]:
#@Sanu: replace the part below by a function --> widely used in our checks (see above)
#summarizing top 5 proton producing/consuming reactions during daytime and nighttime relative to the cytosol --> we did this for all sensitivity checks
#Daytime
fout = open("DaytimeProducing.csv","w")
for i in range(1,7):
    met = ModelF3.metabolites.get_by_id("PROTON_c"+str(i))
    for rxn in met.reactions:
        if round(rxn.flux,3)!=0 and rxn.flux*rxn.metabolites[met]>0:
            fout.write(rxn.id+","+str(((rxn.flux*rxn.metabolites[met])*60*60*2)/1000)+"\n") #convert per sec to per 2h (60*60*2) --> mmol/m-2/s-1
                        
fout.close()

df = pd.read_csv('DaytimeProducing.csv', header=None)
df.rename(columns={0: 'RxnID', 1: 'DaytimeFlux'}, inplace=True)
df['RxnID'] = df['RxnID'].map(lambda x: str(x)[:-1])
df.to_csv('DaytimeProducing.csv', index=False)
dff = df.groupby(['RxnID']).DaytimeFlux.sum().reset_index()
dff.sort_values("DaytimeFlux", axis=0, ascending=False, inplace=True, na_position='first') 
dff.to_csv('DaytimeProducing.csv', index=True)


fout = open("DaytimeConsuming.csv","w")
for i in range(1,7):
    met = ModelF3.metabolites.get_by_id("PROTON_c"+str(i))
    for rxn in met.reactions:
        if round(rxn.flux,3)!=0 and rxn.flux*rxn.metabolites[met]<0:
            fout.write(rxn.id+","+str(((rxn.flux*rxn.metabolites[met])*60*60*2)/1000)+"\n") 
                        
fout.close()

df = pd.read_csv('DaytimeConsuming.csv', header=None)
df.rename(columns={0: 'RxnID', 1: 'DaytimeFlux'}, inplace=True)
df['RxnID'] = df['RxnID'].map(lambda x: str(x)[:-1])
df.to_csv('DaytimeConsuming.csv', index=False)
dff = df.groupby(['RxnID']).DaytimeFlux.sum().reset_index()
dff.sort_values("DaytimeFlux", axis=0, ascending=True, inplace=True, na_position='first') 
dff.to_csv('DaytimeConsuming.csv', index=True)

#Nighttime
fout = open("NighttimeProducing.csv","w")
for i in range(7,13):
    met = ModelF3.metabolites.get_by_id("PROTON_c"+str(i))
    for rxn in met.reactions:
        if round(rxn.flux,3)!=0 and rxn.flux*rxn.metabolites[met]>0:
            fout.write(rxn.id+","+str(((rxn.flux*rxn.metabolites[met])*60*60*2)/1000)+"\n")
                        
fout.close()

df = pd.read_csv('NighttimeProducing.csv', header=None)
df.rename(columns={0: 'RxnID', 1: 'NighttimeFlux'}, inplace=True)
df['RxnID'] = df['RxnID'].map(lambda x: str(x)[:-1])
df['RxnID'] = df['RxnID'].map(lambda x: x.rstrip('1'))
df.to_csv('NighttimeProducing.csv', index=False)
dff = df.groupby(['RxnID']).NighttimeFlux.sum().reset_index()
dff.sort_values("NighttimeFlux", axis=0, ascending=False, inplace=True, na_position='first') 
dff.to_csv('NighttimeProducing.csv', index=True)

fout = open("NighttimeConsuming.csv","w")
for i in range(7,13):
    met = ModelF3.metabolites.get_by_id("PROTON_c"+str(i))
    for rxn in met.reactions:
        if round(rxn.flux,3)!=0 and rxn.flux*rxn.metabolites[met]<0:
            fout.write(rxn.id+","+str(((rxn.flux*rxn.metabolites[met])*60*60*2)/1000)+"\n")
                        
fout.close()

df = pd.read_csv('NighttimeConsuming.csv', header=None)
df.rename(columns={0: 'RxnID', 1: 'NighttimeFlux'}, inplace=True)
df['RxnID'] = df['RxnID'].map(lambda x: str(x)[:-1])
df['RxnID'] = df['RxnID'].map(lambda x: x.rstrip('1'))
df.to_csv('NighttimeConsuming.csv', index=False)
dff = df.groupby(['RxnID']).NighttimeFlux.sum().reset_index()
dff.sort_values("NighttimeFlux", axis=0, ascending=True, inplace=True, na_position='first') 
dff.to_csv('NighttimeConsuming.csv', index=True)

In [ ]:
#Sensitivity analysis - Flux variability analysis
#@Sanu: add FVA part here (see previous notebook on refinement and simulations)? On modelF3 (main ME model)